In [2]:
from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedKFold
from sklearn.metrics import balanced_accuracy_score, precision_recall_curve, confusion_matrix, classification_report, f1_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#загрузка данных

file_list = ['learning_2.csv']
data_set = pd.concat([pd.read_csv(f) for f in file_list], ignore_index=True)
data_set





#iris = datasets.load_iris()
#iris_df = pd.DataFrame(iris.data, columns= iris.feature_names)
#iris_df['target'] = iris.target
#iris_df['name'] = iris_df.target.apply(lambda x : iris.target_names[x])
#iris_df

,DEPT,PS,PZ,GK,NGK,DS,LIT
0,826.2,61.38,6.65,12.58,1.127,0.2140,18
1,826.4,58.82,6.39,12.05,1.160,0.2190,18
2,826.6,59.46,7.16,11.43,1.191,0.2295,18
3,826.8,58.82,8.44,10.97,1.223,0.2450,18
4,827.0,56.90,10.23,10.59,1.250,0.2500,18
...,...,...,...,...,...,...,...
245,875.2,36.76,67.00,4.83,1.655,0.1980,91
246,875.4,37.72,73.01,4.83,1.682,0.1970,91
247,875.6,38.04,75.44,4.78,1.697,0.1980,53
248,875.8,38.68,85.67,4.91,1.688,0.2000,53


In [4]:
X = data_set[['PS', 'PZ', 'GK', 'NGK', 'DS']]
Y = data_set['LIT']

In [5]:
#разбиваем набор данных на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [6]:
#Проведем предварительную обработку данных
#MinMaxScaler сдвигает данные таким образом, что все признаки находились строго в диапазоне от 0 до 1
scaler = MinMaxScaler()
#подгоняем scaler на обучающем наборе данных и масштабируем обучающую и тестовую выборку
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("Отмасштабированная обучающая выборка:\n{}".format(X_train_scaled[:5]))
print("Отмасштабированная тестовая выборка:\n{}".format(X_test_scaled[:5]))

Отмасштабированная обучающая выборка:
[[0.39684814 0.51691564 0.15441176 0.80057389 0.00343643]
 [0.21370583 0.73879613 0.01348039 0.68292683 0.03780069]
 [0.         0.44661687 0.07843137 0.66857963 0.03092784]
 [0.47325692 0.3427065  0.80269608 0.6025825  0.05360825]
 [0.64875836 0.58710457 0.13848039 0.7374462  0.00687285]]
Отмасштабированная тестовая выборка:
[[0.60291309 0.51691564 0.14460784 0.67718795 0.01030928]
 [0.43505253 0.07304482 0.70833333 0.32281205 0.12714777]
 [0.54202483 0.02811951 0.76470588 0.18794835 0.04123711]
 [0.30539637 0.02251757 0.85661765 0.13199426 0.06872852]
 [0.38920726 0.12642794 0.78676471 0.27833572 0.07216495]]


In [10]:
#решетчатый поиск
mlp = MLPRegressor(max_iter=100000000, activation='logistic', solver='sgd', learning_rate='adaptive', random_state=42)
cvc = RepeatedKFold(n_splits=10, n_repeats=5, random_state=42)
#словарь параметров для перекрестной проверки
param_grid = {'alpha': [0.1, 1], 'hidden_layer_sizes': [(40, ), (50, ), (60, )]}
#перекрестная проверка и подбор лучших параметров
grid_search = GridSearchCV(mlp, param_grid, cv=cvc, verbose=5, n_jobs=-1)


In [11]:
#обучение модели
grid_search.fit(X_train_scaled, y_train)

Fitting 50 folds for each of 6 candidates, totalling 300 fits


GridSearchCV(cv=RepeatedKFold(n_repeats=5, n_splits=10, random_state=42),
             estimator=MLPRegressor(activation='logistic',
                                    learning_rate='adaptive',
                                    max_iter=100000000, random_state=42,
                                    solver='sgd'),
             n_jobs=-1,
             param_grid={'alpha': [0.1, 1],
                         'hidden_layer_sizes': [(40,), (50,), (60,)]},
             verbose=5)

In [12]:
print("Наилучшие значения параметров: {}".format(grid_search.best_params_))
print("Наилучшее значение кросс-валидационной правильности: {:.2f}".format(grid_search.best_score_))

Наилучшие значения параметров: {'alpha': 1, 'hidden_layer_sizes': (50,)}
Наилучшее значение кросс-валидационной правильности: 0.75


In [55]:
#запоминаем лучшее параметры модели, подобранные на предыдущем шаге
best_parameters = grid_search.best_params_
#заново строим и обучаем модель уже на лучших параметрах
mlp = MLPRegressor(max_iter=100000000, random_state=42, **best_parameters)
mlp.fit(X_train_scaled, y_train)
#оцениваем качество модели на отмасштабированном тестовом наборе
test_score = mlp.score(X_test_scaled, y_test)
#предсказываем класс для каждого элемента отмасштабированного тестового набора данных
prediction = mlp.predict(X_test_scaled)
print("Правильность на тестовом наборе с наилучшими параметрами R^2:\n{}".format(test_score))
print("Прогнозы:\n{}".format(prediction))


Правильность на тестовом наборе с наилучшими параметрами R^2:
0.5845508070420029
Прогнозы:
[ 15.07675253  54.4156549   78.4897741   17.11330176  15.99954978
  47.33791499  32.69776854 127.07736949  31.00030243  17.60059619
  82.80085522  79.9564905   57.50399941 113.05877126  45.45339983
  41.73080548  85.11662546  29.50793583  52.43599749  86.77759716
  66.31271434  47.68561561  72.21531566  47.37434283  85.33152504
  19.03138984  83.17617107  70.24878374  73.55490019  54.05327569]


In [56]:
#загружаем новый данные
file_list = ['learning.csv']
data_set = pd.concat([pd.read_csv(f) for f in file_list], ignore_index=True)
X_new = data_set[['PS', 'PZ', 'GK', 'NGK', 'DS']]
X_new_scaled = scaler.transform(X_new)
y_new = data_set[['DEPT']]
print("Отмасштабированная выборка новых данных:\n{}".format(X_new_scaled[:5]))

#предсказываем класс для каждого элемента отмасштабированного нового набора данных
prediction_new = mlp.predict(X_new_scaled)
new_score = mlp.score(X_test_scaled, y_test)
#предсказываем вероятность выбора того или иного класса (чем больше число, тем вероятнее этот класс)
print("Прогнозы для новых данных:\n{}".format(prediction_new[:20]))
print("Спрогнозированные вероятности для новых данных:\n{}".format(new_score))

Отмасштабированная выборка новых данных:
[[0.         0.30943513 1.         0.67342799 0.00662252]
 [0.01430487 0.30136561 0.95366795 0.64908722 0.02649007]
 [0.01430487 0.28553693 0.90733591 0.65720081 0.03311258]
 [0.01430487 0.26970826 0.86229086 0.68762677 0.01986755]
 [0.01430487 0.26970826 0.83526384 0.72008114 0.        ]]
Прогнозы для новых данных:
[ 91.03631706  85.56242904  87.71768937  86.80588726  85.21341302
  80.63223843  82.03465891  89.02790845 106.97762492 113.05877126
  94.75188084  77.62472372  79.9564905   71.64425671  68.41549711
  71.54825678  80.37958637  68.57074696  54.4156549   45.45339983]
Спрогнозированные вероятности для новых данных:
0.5845508070420029
